<a href="https://colab.research.google.com/github/audreyvargas314/GWAS-IrisPlex/blob/main/GWASProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files 
uploaded = files.upload()
snps = uploaded['CombinedSNPs.txt'].decode("utf-8")
print(snps) ## upload SNPs into Google Colaboratory and storing it in a variable called "snps"

Saving CombinedSNPs.txt to CombinedSNPs.txt
﻿#CHROM        POS        ID        REF        ALT        QUAL        FILTER        INFO        FORMAT        HG00096        HG00097        HG00099        HG00100        HG00101        HG00102        HG00103        HG00105        HG00106        HG00107        HG00108        HG00109        HG00110        HG00111        HG00112        HG00113        HG00114        HG00115        HG00116        HG00117        HG00118        HG00119        HG00120        HG00121        HG00122        HG00123        HG00125        HG00126        HG00127        HG00128        HG00129        HG00130        HG00131        HG00132        HG00133        HG00136        HG00137        HG00138        HG00139        HG00140        HG00141        HG00142        HG00143        HG00145        HG00146        HG00148        HG00149        HG00150        HG00151        HG00154        HG00155        HG00157        HG00158        HG00159        HG00160        HG00171        HG0017

In [ ]:
import pandas as pd   ## Importing the pandas package to create a DataFrame that will organize SNPs as seen below
import io     ## Importing io for stream handling

df = pd.read_csv(io.StringIO(snps), delim_whitespace=True)
df



,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,HG00107,HG00108,HG00109,HG00110,HG00111,HG00112,HG00113,HG00114,HG00115,HG00116,HG00117,HG00118,HG00119,HG00120,HG00121,HG00122,HG00123,HG00125,HG00126,HG00127,HG00128,HG00129,...,NA21097,NA21098,NA21099,NA21100,NA21101,NA21102,NA21103,NA21104,NA21105,NA21106,NA21107,NA21108,NA21109,NA21110,NA21111,NA21112,NA21113,NA21114,NA21115,NA21116,NA21117,NA21118,NA21119,NA21120,NA21122,NA21123,NA21124,NA21125,NA21126,NA21127,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,5,33951693,rs16891982,C,G,100,PASS,AC=1377;AF=0.27496;AN=5008;NS=2504;DP=19586;EA...,GT,1|1,1|1,1|0,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,0|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,0|1,1|1,...,0|1,1|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,1|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,6,396321,rs12203592,C,T,100,PASS,AC=184;AF=0.0367412;AN=5008;NS=2504;DP=21644;E...,GT,1|0,0|0,1|1,0|0,1|0,0|0,0|1,0|1,0|0,0|1,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,1|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,14,92773663,rs12896399,G,T,100,PASS,AC=1306;AF=0.260783;AN=5008;NS=2504;DP=19551;E...,GT,1|0,1|0,0|0,1|1,0|0,0|1,0|0,0|0,0|1,0|0,0|1,0|1,0|0,0|0,0|0,0|1,0|0,1|0,0|0,0|1,0|1,1|1,0|0,1|0,1|1,0|0,1|1,0|0,1|0,0|0,0|1,...,0|0,1|0,0|1,1|1,1|1,0|1,0|1,1|0,1|0,1|1,0|1,1|0,0|0,0|0,1|0,0|0,1|1,1|1,0|1,0|1,1|1,0|0,0|0,1|1,1|0,0|0,1|1,0|0,0|0,1|0,0|1,1|0,1|0,0|0,1|0,0|1,0|0,0|0,0|1,1|0
3,11,100864447,rs139333502,A,C,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=16332;E...,GT,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,15,28365618,rs12913832,A,G,100,PASS,AC=888;AF=0.177316;AN=5008;NS=2504;DP=19161;EA...,GT,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,0|1,1|1,1|1,1|1,1|1,1|1,0|0,1|1,1|0,1|1,1|1,1|1,1|0,1|1,1|1,1|1,1|1,0|1,1|1,0|1,1|1,0|1,...,0|0,0|0,1|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0
5,15,28230318,rs1800407,C,T,100,PASS,AC=127;AF=0.0253594;AN=5008;NS=2504;DP=19816;E...,GT,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [ ]:

for column in df.columns[9:]:		## iterate through every column in df starting at the first person (column 9)
	for row in range(6):		## iterate through every row to track the reference and alternative alleles
		if df.at[row, column] == "1|1":
			df.at[row, column] = df.at[row, "ALT"] + "|" + df.at[row, "ALT"]	
		elif df.at[row, column] == "0|0":
			df.at[row, column] = df.at[row, "REF"] + "|" + df.at[row, "REF"]
		else:
			df.at[row, column] = df.at[row, "REF"] + "|" + df.at[row, "ALT"]	## the ordering should not matter because it is one 1 and one 0 either way (same value)
## at this point, the DataFrame should have converted all of the data in binary form into the reference and alternate alleles

ma = ["C", "T", "A", "G", "A", "T"]
for column in df.columns[9:]:
	for row in range(6):				## iterating once again
		allele = df.at[row, column]		## setting a variable equal to the current value that will be used in the later method
		if allele.find(ma[row]) == -1:	## .find() was used to check if the minor allele was even in the value 
			df.at[row, column] = 0		## sets data value to 0 if not found
		elif allele.count(ma[row]) == 1:		## using .count() to see the frequency that a substring is found within a string (string is stored in allele variable)
			df.at[row, column] = 1
		else:
			df.at[row, column] = 2		## if not 0 or 1, then it has to be 2
df
## at this point, all of the individuals' data should have stored a 0, 1, or 2


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,HG00107,HG00108,HG00109,HG00110,HG00111,HG00112,HG00113,HG00114,HG00115,HG00116,HG00117,HG00118,HG00119,HG00120,HG00121,HG00122,HG00123,HG00125,HG00126,HG00127,HG00128,HG00129,...,NA21097,NA21098,NA21099,NA21100,NA21101,NA21102,NA21103,NA21104,NA21105,NA21106,NA21107,NA21108,NA21109,NA21110,NA21111,NA21112,NA21113,NA21114,NA21115,NA21116,NA21117,NA21118,NA21119,NA21120,NA21122,NA21123,NA21124,NA21125,NA21126,NA21127,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,5,33951693,rs16891982,C,G,100,PASS,AC=1377;AF=0.27496;AN=5008;NS=2504;DP=19586;EA...,GT,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,1,1,1,2,2,2,2,2,2,2,2,2,2,1,2,1,2,2,2,1,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2
1,6,396321,rs12203592,C,T,100,PASS,AC=184;AF=0.0367412;AN=5008;NS=2504;DP=21644;E...,GT,1,0,2,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,14,92773663,rs12896399,G,T,100,PASS,AC=1306;AF=0.260783;AN=5008;NS=2504;DP=19551;E...,GT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,11,100864447,rs139333502,A,C,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=16332;E...,GT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15,28365618,rs12913832,A,G,100,PASS,AC=888;AF=0.177316;AN=5008;NS=2504;DP=19161;EA...,GT,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,1,0,0,0,1,0,0,0,0,1,0,1,0,1,...,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
5,15,28230318,rs1800407,C,T,100,PASS,AC=127;AF=0.0253594;AN=5008;NS=2504;DP=19816;E...,GT,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
import math 
import numpy as np

alpha = 3.94 
alpha2 = 0.65
##myData = { 'Chromosome 5 B1': [-1.30], 'Chromosome 5 B2': [-0.50], 'Chromosome 6 B1': [0.70], 'Chromsome 6 B2' : [0.73], 'Chromosome 11 B1': [0.47], 'Chromosome 11 B2': [0.27] , 'Chromosome 14 B1': [-0.58], 'Chromosome 14 B2': [-0.03], 'Chromosome 15 B1': [-4.81], 'Chromosome 15 B2' : [-1.79], 'Chromosome 15 b B1': [1.40], 'Chromosome 15 b B2': [0.87] }

##values = myData.values()
##values_list = list(values)
##values_list

values = [-1.30, -0.50, 0.70, 0.73, 0.47, 0.27, -0.58, -0.03, -4.81, -1.79, 1.40, 0.87]
sumb1 = [values[0], values[2], values[4], values[6], values[8], values[10]]
sumb2 = [values[1], values[3], values[5], values[7], values[9], values[11]]


## b-values are now stored in a list in accordance to the ordering of the rows in the DataFrame
def blueProb():
  blue = 0.0            ## end value - probability of a person having blue eyes
  sum1 = 0.0             
  sum2 = 0.0             
  for column in df.columns[9:]:
    for row in range(6):
      sum1 += (float(df.at[row, column])) * (sumb1[row])
      sum2 += float(df.at[row, column]) * (sumb2[row])
      if row == 5:
        blue = (np.exp(alpha + sum1)) / (1 + (np.exp(alpha + sum1)) + (np.exp(alpha2 + sum2)))
        individuals_new.at["Blue", column] = blue
        sum1 = 0.0
        sum2 = 0.0
  

def otherProb():
  other = 0.0                 ## end value - probability of a person having blue eyes
  sum1 = 0.0             
  sum2 = 0.0             
  for column in df.columns[9:]:
    for row in range(6):
      sum1 += (float(df.at[row, column])) * (sumb1[row])
      sum2 += float(df.at[row, column]) * (sumb2[row])
      if row == 5:
        other = (np.exp(alpha2 + sum2)) / (1 + (np.exp(alpha + sum1)) + (np.exp(alpha2 + sum2)))
        individuals_new.at["Other", column] = other
        sum1 = 0.0
        sum2 = 0.0

blueProb()
otherProb()

def brownProb():
  for column in df.columns[9:]:
    for row in range(6):
      individuals_new.at["Brown", column] = (1.0) - (individuals_new.at["Blue", column]) - (individuals_new.at["Other", column])
  
brownProb()

def call():
  blueeyes = 0
  browneyes = 0
  othereyes = 0
  for column in individuals_new.columns[0:]:
      if individuals_new.at["Blue", column] > 0.7:
          blueeyes += 1
      if individuals_new.at["Brown", column] > 0.7:
          browneyes += 1
      if individuals_new.at["Other", column] > 0.7:
          othereyes += 1
  print("Number of people with blue eyes: " + str(blueeyes))
  print("Number of people with brown eyes: " + str(browneyes))
  print("Number of people with other eye color: " + str(othereyes))


call()
individuals_new

NameError: ignored

In [ ]:
##sum += float(x * (float((str(values_list[i])[1:-1]))))
values = [-1.30, -0.50, 0.70, 0.73, 0.47, 0.27, -0.58, -0.03, -4.81, -1.79, 1.40, 0.87]
values
sumb1 = values[0] + values[2] + values[4] + values[6] + values[8] + values[10]
sumb2 = values[1] + values[3] + values[5] + values[7] + values[9] + values[11]
print(sumb1)
print(sumb2)
##+ float(str(values_list[2]))[1:-1] + float(str(values_list[4]))[1:-1] + float(str(values_list[6]))[1:-1] + float(str(values_list[8]))[1:-1] + float(str(values_list[10]))[1:-1]

-4.119999999999999
-0.45000000000000007


In [ ]:
from google.colab import files 
uploaded = files.upload()
people = uploaded['people.txt'].decode("utf-8")



Saving people.txt to people (1).txt


In [ ]:
import pandas as pd  
import io     ## Importing io for stream handling

individuals = pd.read_csv(io.StringIO(people), delim_whitespace=True)
individuals_new = individuals.rename(index={0: 'Blue', 1 : "Brown", 2 : "Other"})
individuals_new

,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,HG00107,HG00108,HG00109,HG00110,HG00111,HG00112,HG00113,HG00114,HG00115,HG00116,HG00117,HG00118,HG00119,HG00120,HG00121,HG00122,HG00123,HG00125,HG00126,HG00127,HG00128,HG00129,HG00130,HG00131,HG00132,HG00133,HG00136,HG00137,HG00138,HG00139,HG00140,...,NA21097,NA21098,NA21099,NA21100,NA21101,NA21102,NA21103,NA21104,NA21105,NA21106,NA21107,NA21108,NA21109,NA21110,NA21111,NA21112,NA21113,NA21114,NA21115,NA21116,NA21117,NA21118,NA21119,NA21120,NA21122,NA21123,NA21124,NA21125,NA21126,NA21127,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
Blue,1|1,1|1,1|0,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,0|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,0|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,1|1,...,0|1,1|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,1|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
Brown,1|0,0|0,1|1,0|0,1|0,0|0,0|1,0|1,0|0,0|1,1|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,1|0,1|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|1,0|0,0|0,0|1,0|0,0|1,0|0,0|1,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
Other,1|0,1|0,0|0,1|1,0|0,0|1,0|0,0|0,0|1,0|0,0|1,0|1,0|0,0|0,0|0,0|1,0|0,1|0,0|0,0|1,0|1,1|1,0|0,1|0,1|1,0|0,1|1,0|0,1|0,0|0,0|1,1|0,0|1,0|0,1|1,0|1,0|1,0|0,0|0,1|0,...,0|0,1|0,0|1,1|1,1|1,0|1,0|1,1|0,1|0,1|1,0|1,1|0,0|0,0|0,1|0,0|0,1|1,1|1,0|1,0|1,1|1,0|0,0|0,1|1,1|0,0|0,1|1,0|0,0|0,1|0,0|1,1|0,1|0,0|0,1|0,0|1,0|0,0|0,0|1,1|0


In [ ]:
def call():
  blue = blueProb()
  brown = brownProb()
  other = otherProb()
  blueeyes = 0
  browneyes = 0
  othereyes = 0
  ## make our table here
  c = 0
  for column in blue.columns[9:]:
    for row in range(6):
      if blue.at[row, column] > 0.7:
          blueeyes += 1
      individuals_new[0, c] = blue.at[row, column]
      c += 1

  for column in brown.columns[9:]:
    for row in range(6):
      if brown.at[row, column] > 0.7:
          browneyes += 1
      individuals_new[1, c] = brown.at[row, column]
      c += 1

  for column in other.columns[9:]:
    for row in range(6):
      if other.at[row, column] > 0.7:
        othereyes +=1
      individuals_new[2, c] = other.at[row,column]
      c += 1

print("Number of people with blue eyes: " + blueeyes)
print("Number of people with brown eyes: " + browneyes)
print("Number of people with other eye color: " + othereyes)
        ## go through other DataFrame
## print("Number of people with blue eyes: " + blueeyes)
## print number of brown-eyed people using ^^
## print number of other-eyed people using ^^^
individuals_new

Project Outline: 
Audrey's code runs first. Her code consists of translating the reference/alternative alleles to 0s, 1s, and 2s (minor alleles). Then, she returns the new dataframe with the 0s, 1s, and 2s to Emily. Emily will then take that info and call my function to receive the probabilities for each eye color (brown, blue, and other) for each individual. Sriya's code will be her taking in what Audrey created (0s, 1s, and 2s). From there, Emily will compare to see if any of the probabilities are greater than 0.7 for each individual. Emily will then create a table that show's each individual's probability for each eye color. 